In [42]:
library(dplyr, warn.conflicts = FALSE)
library(DBI)
library(stringdist)
library(tidytext)


normalize_name_string <- function(name) {

    title_normed <- stringr::str_match_all(stringr::str_to_lower(name), '[a-z0-9]')[[1]][,1] %>% paste0(collapse = "")

    return(title_normed)

}



get_name_similarity_index <- function(name_1, name_2) {

    reduced_name_1 <- normalize_name_string(name_1)

    reduced_name_2 <- normalize_name_string(name_2)

    max_len <- max(nchar(reduced_name_1), nchar(reduced_name_2))

    ratio <- (max_len - adist(reduced_name_1, reduced_name_2))/max_len

    return(ratio)

}



get_one_bag_df <- function(match_df) {

    l_1 <- stringr::str_match_all(match_df$company_name, '[^\\s]+')
    l_2 <- stringr::str_match_all(match_df$issuer_name, '[^\\s]+')

    min_l <- unlist(lapply(1:nrow(match_df), function(i) {min(nrow(l_1[[i]]), nrow(l_2[[i]]))}))

    vec_1 <- c()
    vec_2 <- c()

    for(i in 1:length(min_l)) {

        vec_1 <- c(vec_1, l_1[[i]][1:min_l[i], 1])
        vec_2 <- c(vec_2, l_2[[i]][1:min_l[i], 1])

    }

    df <- data.frame(word1 = vec_1, word2 = vec_2, stringsAsFactors = FALSE)


    return(df)

}




In [70]:
Sys.setenv(PGHOST = '10.101.13.99')
Sys.setenv(PGDATABASE = 'crsp')
Sys.setenv(PGUSER = 'bdcallen')
Sys.setenv(PGPASSWORD = 'temp_20180308')

In [92]:
Sys.getenv('PGDATABASE')

[1] "crsp"

In [91]:
pg <- dbConnect(RPostgreSQL::PostgreSQL())

In [95]:
cusip_cik <- tbl(pg, sql("SELECT * FROM edgar.cusip_cik_test"))

In [96]:
cusip_cik %>% group_by(cik, cusip) %>% summarise(freq = n()) %>% ungroup() %>% inner_join(cusip_cik) %>% count()

cusip_cik %>% count()

cusip_cik %>% filter(!is.na(cusip)) %>% count()

Joining, by = c("cik", "cusip")


# Source:   lazy query [?? x 1]
# Database: postgres 11.0.9 [bdcallen@10.101.13.99:5432/crsp]
        n
    <dbl>
1 1262828

# Source:   lazy query [?? x 1]
# Database: postgres 11.0.9 [bdcallen@10.101.13.99:5432/crsp]
        n
    <dbl>
1 1335995

# Source:   lazy query [?? x 1]
# Database: postgres 11.0.9 [bdcallen@10.101.13.99:5432/crsp]
        n
    <dbl>
1 1262828

In [94]:
issuers <- tbl(pg, sql('SELECT * FROM cusipm.issuer')) %>% collect()
issuers <- issuers %>% rename(cusip6 = issuer_num) # change issuer_num to cusip6



# Annoying to have the company names (issuer_name_1, 2 and 3) and their name histories (issuer_adl_1, 2, 3 and 4) split over
# several fields. Amalgamate them to form issuer_name and issuer_adl respectively

issuers <- issuers %>%
    mutate(issuer_name = ifelse(is.na(issuer_name_2), issuer_name_1, ifelse(is.na(issuer_name_3), 
    paste(issuer_name_1, issuer_name_2), paste(issuer_name_1, issuer_name_2, issuer_name_3)))) 


issuers <- issuers %>%
    mutate(issuer_adl = ifelse(is.na(issuer_adl_1), NA, issuer_adl_1)) %>%
    mutate(issuer_adl = ifelse(is.na(issuer_adl_2), issuer_adl, paste(issuer_adl, issuer_adl_2))) %>%
    mutate(issuer_adl = ifelse(is.na(issuer_adl_3), issuer_adl, paste(issuer_adl, issuer_adl_3))) %>%
    mutate(issuer_adl = ifelse(is.na(issuer_adl_4), issuer_adl, paste(issuer_adl, issuer_adl_4))) 

In [97]:
issuers %>% select(cusip6, issuer_name, issuer_adl) %>% head(10)

cusip6,issuer_name,issuer_adl
000021,A & A INTL INDS INC,FORMERLY A AND A FOODS LTD TO 08/05/1997
000024,A & B TRANSN SERVICES INC,NA
000027,A & E CAP FDG INC,NA
00002T,A&B GEOSCIENCE CORP,NAME CHANGED TO ARAWAK ENERGY CORP 05/05/2003 SEE 038743
000032,A & E PLASTIK PAK INC,CONVERTED TO PRIVATE CO 06/22/1978
000060,A& ECONOMY LODGING ASSOC,NA
00007T,A & F ENTMT CORP,NA
000096,A & F INVESTING CO,NA
0000CG,CGA MINING LTD CDS-,NA
000101,A & J VENTURE CAP GROUP INC,NAME CHANGED TO RELIABRAND INC 02/24/2011 SEE 75944L


In [43]:
# This cell takes all matches with cusipm.issuer from the cases with valid 9 digit cusips, then creates map_df


m9_issuers <- cusip_cik %>% distinct(cik, cusip, check_digit, company_name) %>% collect() %>%
    filter(nchar(cusip) == 9 & substr(cusip, 9, 9) == as.character(check_digit)) %>%
    mutate(cusip6 = substr(cusip, 1, 6)) %>%
    inner_join(issuers, by = 'cusip6')

m9_issuers <- m9_issuers %>%
    mutate(issuer_name = ifelse(is.na(issuer_name_2), issuer_name_1,
    ifelse(is.na(issuer_name_3), paste(issuer_name_1, issuer_name_2), paste(issuer_name_1, issuer_name_2, issuer_name_3))))

m9_issuers <- m9_issuers %>%
        mutate(issuer_adl = ifelse(is.na(issuer_adl_1), NA, issuer_adl_1)) %>%
        mutate(issuer_adl = ifelse(is.na(issuer_adl_2), issuer_adl, paste(issuer_adl, issuer_adl_2))) %>%
        mutate(issuer_adl = ifelse(is.na(issuer_adl_3), issuer_adl, paste(issuer_adl, issuer_adl_3))) %>%
        mutate(issuer_adl = ifelse(is.na(issuer_adl_4), issuer_adl, paste(issuer_adl, issuer_adl_4)))



m9_issuers <- m9_issuers %>% mutate(company_name_raw = stringr::str_to_upper(company_name),
                                    issuer_name_raw = stringr::str_to_upper(issuer_name)) %>%
    mutate(company_name_raw = gsub('[^A-Z0-9\\s]', ' ', company_name_raw),
           issuer_name_raw = gsub('[^A-Z0-9\\s]', ' ', issuer_name_raw)) %>%
    mutate(company_name_raw = gsub('\\s+', ' ', company_name_raw),
           issuer_name_raw = gsub('\\s+', ' ', issuer_name_raw)) %>%
    mutate(company_name_raw = gsub('\\s+', ' ', company_name_raw),
           issuer_name_raw = gsub('\\s+', ' ', issuer_name_raw)) %>%
    mutate(company_name_raw = gsub('^\\s', '', company_name_raw),
           issuer_name_raw = gsub('^\\s', '', issuer_name_raw)) %>%
    mutate(company_name_raw = gsub('\\s$', '', company_name_raw), issuer_name_raw = gsub('\\s$', '', issuer_name_raw))


m9_issuers$sim_index_raw <- unlist(lapply(1:nrow(m9_issuers),
        function(i) {get_name_similarity_index(m9_issuers$company_name_raw[i], m9_issuers$issuer_name_raw[i])}))


one_bag_df <- get_one_bag_df(m9_issuers %>% filter(sim_index_raw >= 0.6) %>%
                                 select(company_name_raw, issuer_name_raw) %>%
                                 rename(company_name = company_name_raw, issuer_name = issuer_name_raw))

one_bag_df %>% filter(word1 != word2 & nchar(word1) > 1 & nchar(word2) > 1 &
    substr(word1, 1, 1) == substr(word2, 1, 1) & nchar(word1) > nchar(word2) &
    ifelse(substr(word1, nchar(word1), nchar(word1)) == 'S', substr(word2, nchar(word2), nchar(word2)) == 'S', TRUE)) %>%
    group_by(word1, word2) %>% summarise(freq = n()) %>% arrange(desc(freq)) %>% print(n=20)


bad_maps <- c(181, 198, 213, 218, 219, 227, 228, 238, 241, 242)

map_df <- one_bag_df %>% filter(word1 != word2 & nchar(word1) > 1 & nchar(word2) > 1 &
        substr(word1, 1, 1) == substr(word2, 1, 1) & nchar(word1) > nchar(word2) &
    ifelse(substr(word1, nchar(word1), nchar(word1)) == 'S', substr(word2, nchar(word2), nchar(word2)) == 'S', TRUE)) %>%
    group_by(word1, word2) %>% summarise(freq = n()) %>%
    filter(freq >= 3) %>% arrange(desc(freq)) %>% ungroup() %>% slice(-bad_maps) %>% select(word1, word2)


one_bag_df %>% filter(word1 != word2 & nchar(word1) > 1 & nchar(word2) > 1 &
    substr(word1, 1, 1) == substr(word2, 1, 1) & nchar(word1) > nchar(word2) &
    ifelse(substr(word1, nchar(word1), nchar(word1)) == 'S', substr(word2, nchar(word2), nchar(word2)) == 'S', TRUE)) %>%
    group_by(word1, word2) %>% summarise(freq = n()) %>%
    filter(freq >= 3) %>% arrange(desc(freq)) %>% ungroup() %>% slice(bad_maps)


map_df$regex1 <- paste0('(?:\\s|^)', map_df$word1, '(?:\\s|$)')
map_df$regex2 <- paste0(' ', map_df$word2, ' ')


m9_issuers$company_name_norm <- stringi::stri_replace_all_regex(m9_issuers$company_name_raw,
                                                                map_df$regex1, map_df$regex2, vectorize_all = FALSE) %>%
    gsub('^\\s', '', .) %>% gsub('\\s$', '', .)

m9_issuers$issuer_name_norm <- stringi::stri_replace_all_regex(m9_issuers$issuer_name_raw,
                                                               map_df$regex1, map_df$regex2, vectorize_all = FALSE) %>%
    gsub('^\\s', '', .) %>% gsub('\\s$', '', .)



m9_issuers %>% select(company_name, company_name_norm, issuer_name, issuer_name_norm) %>% print(n=20)



m9_issuers$sim_index_norm <- unlist(lapply(1:nrow(m9_issuers),
            function(i) {get_name_similarity_index(m9_issuers$company_name_norm[i], m9_issuers$issuer_name_norm[i])}))

Joining, by = c("cik", "cusip")


# Source:   lazy query [?? x 1]
# Database: postgres 11.0.8 [bdcallen@10.101.13.99:5432/crsp]
        n
    <dbl>
1 1262828

# Source:   lazy query [?? x 1]
# Database: postgres 11.0.8 [bdcallen@10.101.13.99:5432/crsp]
        n
    <dbl>
1 1335995

# Source:   lazy query [?? x 1]
# Database: postgres 11.0.8 [bdcallen@10.101.13.99:5432/crsp]
        n
    <dbl>
1 1262828

# A tibble: 500 x 3
# Groups:   word1 [465]
           word1 word2  freq
           <chr> <chr> <int>
 1          FUND    FD  1385
 2         TRUST    TR  1313
 3      HOLDINGS HLDGS   967
 4     FINANCIAL  FINL   826
 5       SYSTEMS   SYS   635
 6 INTERNATIONAL  INTL   634
 7     MUNICIPAL   MUN   584
 8      SERVICES  SVCS   462
 9    INDUSTRIES  INDS   426
10     RESOURCES   RES   416
11       CAPITAL   CAP   373
12       HOLDING  HLDG   234
13        REALTY  RLTY   225
14       MEDICAL   MED   222
15    PROPERTIES PPTYS   179
16 ENTERTAINMENT ENTMT   178
17      PRODUCTS PRODS   173
18    INVESTMENT  INVT   163
19         POWER   PWR   163
20      DIVIDEND DIVID   156
# ... with 480 more rows


word1,word2,freq
INCOME,II,5
CORP,CAP,4
PUBLIC,PLC,4
COM,CO,3
DEUTSCHE,DWS,3
INC\,INC,3
INSURED,INVT,3
SIMPLETECH,SIMPLE,3
TERM,TR,3
TREE,TR,3


# A tibble: 40,256 x 4
               company_name     company_name_norm                  issuer_name
                      <chr>                 <chr>                        <chr>
 1 K TRON INTERNATIONAL INC       K TRON INTL INC FIRST FID BANCORPORATION NEW
 2 K TRON INTERNATIONAL INC       K TRON INTL INC              K TRON INTL INC
 3                 AAR CORP                AAR CO                     AAR CORP
 4      TRANZONIC COMPANIES         TRANZONIC COS                TRANZONIC COS
 5      TRANZONIC COMPANIES         TRANZONIC COS                TRANZONIC COS
 6      ABBOTT LABORATORIES           ABBOTT LABS                  ABBOTT LABS
 7 ABERDEEN IDAHO MINING CO ABERDEEN IDAHO MNG CO              ABERDEEN MNG CO
 8    ABRAMS INDUSTRIES INC       ABRAMS INDS INC              ABRAMS INDS INC
 9          SERVIDYNE, INC.         SERVIDYNE INC                SERVIDYNE INC
10               WORLDS INC            WORLDS INC                   WORLDS INC
11           WORLDS COM INC  

In [44]:
map_df

word1,word2,regex1,regex2
FUND,FD,(?:\s|^)FUND(?:\s|$),FD
TRUST,TR,(?:\s|^)TRUST(?:\s|$),TR
HOLDINGS,HLDGS,(?:\s|^)HOLDINGS(?:\s|$),HLDGS
FINANCIAL,FINL,(?:\s|^)FINANCIAL(?:\s|$),FINL
SYSTEMS,SYS,(?:\s|^)SYSTEMS(?:\s|$),SYS
INTERNATIONAL,INTL,(?:\s|^)INTERNATIONAL(?:\s|$),INTL
MUNICIPAL,MUN,(?:\s|^)MUNICIPAL(?:\s|$),MUN
SERVICES,SVCS,(?:\s|^)SERVICES(?:\s|$),SVCS
INDUSTRIES,INDS,(?:\s|^)INDUSTRIES(?:\s|$),INDS
RESOURCES,RES,(?:\s|^)RESOURCES(?:\s|$),RES


In [46]:
m9_issuers %>% distinct(cik, cusip) %>% count()

n
35559


In [47]:
m9_issuers %>% group_by(cik, cusip) %>% summarise(sim_index_raw_max = max(sim_index_raw)) %>%
    ungroup() %>% filter(sim_index_raw_max >= 0.8) %>% count()

n
21672


In [48]:
m9_issuers %>% group_by(cik, cusip) %>% summarise(sim_index_norm_max = max(sim_index_norm)) %>%
    ungroup() %>% filter(sim_index_norm_max >= 0.9) %>% inner_join(m9_issuers) %>% 
    filter(sim_index_norm == sim_index_norm_max) %>%
    select(cik, cusip, company_name, issuer_name, issuer_adl, sim_index_norm)

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,issuer_adl,sim_index_norm
20,482730108,K TRON INTERNATIONAL INC,K TRON INTL INC,NA,1
1750,000361105,AAR CORP,AAR CORP,NA,1
1761,894120104,TRANZONIC COMPANIES,TRANZONIC COS,ACQUIRED BY LINSALATA CAP PARTNERS FD II L P-A PRIVATE CO 02/05/1998,1
1761,894120203,TRANZONIC COMPANIES,TRANZONIC COS,ACQUIRED BY LINSALATA CAP PARTNERS FD II L P-A PRIVATE CO 02/05/1998,1
1800,002824100,ABBOTT LABORATORIES,ABBOTT LABS,NA,1
1923,003788106,ABRAMS INDUSTRIES INC,ABRAMS INDS INC,FORMERLY ABRAMS A R INC TO 01/17/1972 & NAME CHANGED TO SERVIDYNE INC 07/15/2006 SEE 81765M,1
1923,81765M106,"SERVIDYNE, INC.",SERVIDYNE INC,NA,1
1961,98159J101,WORLDS INC,WORLDS INC,NA,1
1961,981918105,WORLDS INC,WORLDS INC,NAME CHANGED TO WORLDS COM INC 12/27/1999 SEE 98156G,1
1969,031095102,AMETECH INC,AMETECH INC,NA,1


In [49]:
valid9s_above_10 <- cusip_cik %>% group_by(cik, cusip) %>% summarise(freq = n()) %>% ungroup() %>% inner_join(cusip_cik) %>%
    filter(nchar(cusip) == 9 & substr(cusip, 9, 9) == as.character(check_digit) & freq >= 10) %>% collect() %>%
    distinct(cik, cusip, company_name)

valid9s_above_10 <- valid9s_above_10 %>% mutate(cusip6 = substr(cusip, 1, 6), ncusip = substr(cusip, 1, 8))


valid9s_above_10_w_issuers <- valid9s_above_10 %>% inner_join(issuers)

valid9s_above_10_w_issuers %>% distinct(cik, cusip) %>% count()


valid9s_above_10_w_issuers <- valid9s_above_10_w_issuers %>%
    mutate(issuer_name = ifelse(is.na(issuer_name_2), issuer_name_1,
    ifelse(is.na(issuer_name_3), paste(issuer_name_1, issuer_name_2), paste(issuer_name_1, issuer_name_2, issuer_name_3))))

valid9s_above_10_w_issuers <- valid9s_above_10_w_issuers %>%
    mutate(issuer_adl = ifelse(is.na(issuer_adl_1), NA, issuer_adl_1)) %>%
    mutate(issuer_adl = ifelse(is.na(issuer_adl_2), issuer_adl, paste(issuer_adl, issuer_adl_2))) %>%
    mutate(issuer_adl = ifelse(is.na(issuer_adl_3), issuer_adl, paste(issuer_adl, issuer_adl_3))) %>%
    mutate(issuer_adl = ifelse(is.na(issuer_adl_4), issuer_adl, paste(issuer_adl, issuer_adl_4)))


valid9s_above_10_w_issuers <- valid9s_above_10_w_issuers %>% mutate(company_name_raw = stringr::str_to_upper(company_name),
                                                issuer_name_raw = stringr::str_to_upper(issuer_name)) %>%
    mutate(company_name_raw = gsub('[^A-Z0-9\\s]', ' ', company_name_raw),
           issuer_name_raw = gsub('[^A-Z0-9\\s]', ' ', issuer_name_raw)) %>%
    mutate(company_name_raw = gsub('\\s+', ' ', company_name_raw),
           issuer_name_raw = gsub('\\s+', ' ', issuer_name_raw)) %>%
    mutate(company_name_raw = gsub('\\s+', ' ', company_name_raw),
           issuer_name_raw = gsub('\\s+', ' ', issuer_name_raw)) %>%
    mutate(company_name_raw = gsub('^\\s', '', company_name_raw),
           issuer_name_raw = gsub('^\\s', '', issuer_name_raw)) %>%
    mutate(company_name_raw = gsub('\\s$', '', company_name_raw), issuer_name_raw = gsub('\\s$', '', issuer_name_raw))


valid9s_above_10_w_issuers$company_name_norm <- stringi::stri_replace_all_regex(valid9s_above_10_w_issuers$company_name_raw,
                                                                      map_df$regex1, map_df$regex2, vectorize_all = FALSE) %>%
    gsub('^\\s', '', .) %>% gsub('\\s$', '', .)

valid9s_above_10_w_issuers$issuer_name_norm <- stringi::stri_replace_all_regex(valid9s_above_10_w_issuers$issuer_name_raw,
                                                                     map_df$regex1, map_df$regex2, vectorize_all = FALSE) %>%
    gsub('^\\s', '', .) %>% gsub('\\s$', '', .)



valid9s_above_10_w_issuers$sim_index_norm <- unlist(lapply(1:nrow(valid9s_above_10_w_issuers),
        function(i) {get_name_similarity_index(valid9s_above_10_w_issuers$company_name_norm[i],
                                               valid9s_above_10_w_issuers$issuer_name_norm[i])}))



valid9s_above_10_w_issuers %>% distinct(cik, cusip) %>% count()

sim_index_max <- valid9s_above_10_w_issuers %>% group_by(cik, cusip) %>%
                    summarise(sim_index_max = max(sim_index_norm)) %>% ungroup()

sim_index_max %>% filter(sim_index_max == 1) %>% count()

sim_index_max %>% filter(sim_index_max >= 0.6 & sim_index_max < 0.8) %>% count()

sim_index_max %>% filter(sim_index_max < 0.3) %>% count()

valid9s_above_10_w_issuers <- valid9s_above_10_w_issuers %>% inner_join(sim_index_max)

Joining, by = c("cik", "cusip")
Joining, by = "cusip6"


n
19321


n
19321


n
15884


n
792


n
177


Joining, by = c("cik", "cusip")


In [51]:
sim_index_max %>% filter(sim_index_max == 1) %>% count()

n
15884


In [55]:
sim_index_max %>% count()

n
19321


In [54]:
sim_index_max %>% filter(sim_index_max >= 0.8) %>% count()

n
18072


In [53]:
sim_index_max %>% filter(sim_index_max >= 0.6 & sim_index_max < 0.8) %>% count()

n
792


In [50]:
valid9s_above_10_w_issuers

cik,cusip,company_name,cusip6,ncusip,issuer_check,issuer_name_1,issuer_name_2,issuer_name_3,issuer_adl_1,...,entry_date,cp_institution_type,issuer_name,issuer_adl,company_name_raw,issuer_name_raw,company_name_norm,issuer_name_norm,sim_index_norm,sim_index_max
1075607,87233Q108,TC PIPELINES LP,87233Q,87233Q10,9,TC PIPELINES LP,NA,NA,NA,...,1999-04-30,NA,TC PIPELINES LP,NA,TC PIPELINES LP,TC PIPELINES LP,TC PIPELINES LP,TC PIPELINES LP,1.0000000,1.0000000
814050,278864202,ECOGEN INC,278864,27886420,4,ECOGEN INC,NA,NA,NA,...,1993-10-31,NA,ECOGEN INC,NA,ECOGEN INC,ECOGEN INC,ECOGEN INC,ECOGEN INC,1.0000000,1.0000000
74273,680293107,OLD STONE CORP,680293,68029310,8,OLD STONE CORP,NA,NA,NA,...,1993-10-31,NA,OLD STONE CORP,NA,OLD STONE CORP,OLD STONE CORP,OLD STONE CO,OLD STONE CO,1.0000000,1.0000000
74273,680293305,OLD STONE CORP,680293,68029330,8,OLD STONE CORP,NA,NA,NA,...,1993-10-31,NA,OLD STONE CORP,NA,OLD STONE CORP,OLD STONE CORP,OLD STONE CO,OLD STONE CO,1.0000000,1.0000000
1013564,307901108,FARM FAMILY HOLDINGS INC,307901,30790110,9,FARM FAMILY HLDGS INC,NA,NA,NA,...,1996-04-23,NA,FARM FAMILY HLDGS INC,NA,FARM FAMILY HOLDINGS INC,FARM FAMILY HLDGS INC,FARM FAMILY HLDGS INC,FARM FAMILY HLDGS INC,1.0000000,1.0000000
887941,379335102,GLOBAL INDUSTRIAL TECHNOLOGIES INC,379335,37933510,3,GLOBAL INDL TECHNOLOGIES INC,NA,NA,ACQUIRED BY RHI AG 12/30/1999,...,1995-10-20,NA,GLOBAL INDL TECHNOLOGIES INC,ACQUIRED BY RHI AG 12/30/1999,GLOBAL INDUSTRIAL TECHNOLOGIES INC,GLOBAL INDL TECHNOLOGIES INC,GLOBAL INDL TECHNOLOGIES INC,GLOBAL INDL TECHNOLOGIES INC,1.0000000,1.0000000
911522,361942105,GST TELECOMMUNICATIONS INC,361942,36194210,6,GST TELECOMMUNICATIONS INC,NA,NA,NA,...,1995-02-22,NA,GST TELECOMMUNICATIONS INC,NA,GST TELECOMMUNICATIONS INC,GST TELECOMMUNICATIONS INC,GST TELECOMMUNICATIONS INC,GST TELECOMMUNICATIONS INC,1.0000000,1.0000000
819521,404150104,VEST H D INC /TX/,404150,40415010,5,H D VEST INC,NA,NA,ACQUIRED BY WELLS FARGO & CO,...,1993-10-31,NA,H D VEST INC,ACQUIRED BY WELLS FARGO & CO NEW 07/03/2001,VEST H D INC TX,H D VEST INC,VEST H D INC TX,H D VEST INC,0.4545455,0.4545455
1063344,427825104,HERSHA HOSPITALITY TRUST,427825,42782510,5,HERSHA HOSPITALITY TR,NA,NA,NA,...,1999-01-07,NA,HERSHA HOSPITALITY TR,NA,HERSHA HOSPITALITY TRUST,HERSHA HOSPITALITY TR,HERSHA HOSPITALITY TR,HERSHA HOSPITALITY TR,1.0000000,1.0000000
1005967,403917107,HMT TECHNOLOGY CORP,403917,40391710,8,HMT TECHNOLOGY CORP,NA,NA,NA,...,1996-01-22,NA,HMT TECHNOLOGY CORP,NA,HMT TECHNOLOGY CORP,HMT TECHNOLOGY CORP,HMT TECHNOLOGY CO,HMT TECHNOLOGY CO,1.0000000,1.0000000


In [98]:
cusip_cik_exceptions <- tbl(pg, sql("SELECT * FROM edgar.cusip_cik_exceptions")) %>% collect()

In [99]:
cusip_cik_exceptions %>% head(10)

cik,cusip,company_name,issuer_name,issuer_adl,company_name_norm,issuer_name_norm,sim_index_norm,sim_index_max,valid_match,...,better_cusip_issuer_name,better_cusip_issuer_adl,sim_ix_better_cusip,better_cusip6,better_cusip6_issuer_name,better_cusip6_issuer_adl,better_cusip8,better_cusip8_comnam,other_reason,cusip_raw
1324105,08883T200,"Bidz.com, Inc.",BIDZ COM INC,NA,NA,NA,NA,NA,TRUE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,8883T200
830736,090649104,BIOPOOL INTERNATIONAL INC,BIOPOOL INTL INC,NAME CHANGED TO XTRANA INC 06/29/2001 SEE 98415F,NA,NA,NA,NA,TRUE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,90649104
54883,048666510,KEANE INC,ATLANTIC KEYSTONE PETE CO LTD,NA,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,48666510
929940,004532710,ASPEN TECHNOLOGY INC /DE/,ACKERLEY GROUP INC,NA,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,04532710
927066,089151A10,DAVITA INC,BIG 5 HLDGS CORP,NA,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,89151A10
817946,045076109,MAYORS JEWELERS INC/DE,ASHTON MNG N L,NA,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,45076109
33073,029403700,VISKASE COMPANIES INC,AMERICAN SVGS BK FSB STOCKTON CALIF,NA,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,29403700
790715,082656103,SIGMA DESIGNS INC,BENTLEY NATL CORP,NAME CHANGED TO SUNSHINE RES INTL INC 09/01/1975 SEE 867858,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,82656103
41850,037935240,GLOBAL MARINE INC,APPLIED MICROSYSTEMS CORP,NA,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,37935240
1094348,029015110,ELOYALTY CORP,AMERICAN PIPE & CONSTR CO,NAME CHANGED TO AMERON INC 01/06/1970,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,29015110


In [101]:
dim(cusip_cik_exceptions)

[1] 4604   24

In [163]:
colnames(cusip_cik_exceptions)

[1] "cik"                       "cusip"                    
 [3] "company_name"              "issuer_name"              
 [5] "issuer_adl"                "company_name_norm"        
 [7] "issuer_name_norm"          "sim_index_norm"           
 [9] "sim_index_max"             "valid_match"              
[11] "better_cik"                "better_cik_company_name"  
[13] "sim_ix_better_cik"         "better_cusip"             
[15] "better_cusip_issuer_name"  "better_cusip_issuer_adl"  
[17] "sim_ix_better_cusip"       "better_cusip6"            
[19] "better_cusip6_issuer_name" "better_cusip6_issuer_adl" 
[21] "better_cusip8"             "better_cusip8_comnam"     
[23] "other_reason"              "cusip_raw"

In [164]:
to_analyse <- valid9s_above_10_w_issuers %>% filter(sim_index_max < 0.8) %>% 
                anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

cik,cusip,company_name,cusip6,ncusip,issuer_check,issuer_name_1,issuer_name_2,issuer_name_3,issuer_adl_1,...,entry_date,cp_institution_type,issuer_name,issuer_adl,company_name_raw,issuer_name_raw,company_name_norm,issuer_name_norm,sim_index_norm,sim_index_max


In [169]:
to_analyse$cusip_raw <- to_analyse$cusip

In [168]:
colnames(to_analyse)[colnames(to_analyse) %in% colnames(cusip_cik_exceptions)]

[1] "cik"               "cusip"             "company_name"     
[4] "issuer_name"       "issuer_adl"        "company_name_norm"
[7] "issuer_name_norm"  "sim_index_norm"    "sim_index_max"

In [170]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

[1] "cik"               "cusip"             "company_name"     
 [4] "issuer_name"       "issuer_adl"        "company_name_norm"
 [7] "issuer_name_norm"  "sim_index_norm"    "sim_index_max"    
[10] "cusip_raw"

In [171]:
colnames(cusip_cik_exceptions)[!colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]

[1] "valid_match"               "better_cik"               
 [3] "better_cik_company_name"   "sim_ix_better_cik"        
 [5] "better_cusip"              "better_cusip_issuer_name" 
 [7] "better_cusip_issuer_adl"   "sim_ix_better_cusip"      
 [9] "better_cusip6"             "better_cusip6_issuer_name"
[11] "better_cusip6_issuer_adl"  "better_cusip8"            
[13] "better_cusip8_comnam"      "other_reason"

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [ ]:
# Note here, in the initial analysis below, ignored normalization when calculating sim_index_norm and sim_index_max for 
# nines_w_stocknames, as the set of valid 9 digit cusips which join stocknames but not issuers is an order of magnitude less
# making normalization less necessary to help narrow the numbers to a manageable level

In [63]:
stocknames <- tbl(pg, sql('SELECT * FROM crsp.stocknames')) %>% collect()

In [105]:
nines_not_in_issuers <- valid9s_above_10 %>% anti_join(valid9s_above_10_w_issuers %>% distinct(cik, cusip))
nines_not_in_issuers <- nines_not_in_issuers %>% mutate(ncusip = substr(cusip, 1, 8))
nines_w_stocknames <- nines_not_in_issuers %>% inner_join(stocknames, by = 'ncusip') %>%
                    select(cik, cusip.x, ncusip, company_name, comnam) %>% rename(issuer_name = comnam)

nines_w_stocknames$sim_index_norm <- unlist(lapply(1:nrow(nines_w_stocknames),
    function(i) {get_name_similarity_index(nines_w_stocknames$company_name[i], nines_w_stocknames$issuer_name[i])}))

nines_w_stocknames <- nines_w_stocknames %>% rename(cusip = cusip.x)

nines_w_stocknames <- nines_w_stocknames %>% inner_join(nines_w_stocknames %>% group_by(cik, cusip) %>%
                    summarise(sim_index_max = max(sim_index_norm)) %>% ungroup())

Joining, by = c("cik", "cusip")
Joining, by = c("cik", "cusip")


In [106]:
nines_w_stocknames %>% head(10)

cik,cusip,ncusip,company_name,issuer_name,sim_index_norm,sim_index_max
1009404,G90751101,G9075110,TRITON ENERGY LTD,TRITON ENERGY LTD,1.0000000,1.0000000
1610682,903318103,90331810,USD Partners LP,U S D PARTNERS LP,1.0000000,1.0000000
1707092,G2709G107,G2709G10,Delphi Technologies PLC,DELPHI TECHNOLOGIES PLC,1.0000000,1.0000000
1100663,46435G250,46435G25,iSHARES TRUST,ISHARES TRUST,1.0000000,1.0000000
1681903,44931Q104,44931Q10,"ICC Holdings, Inc.",I C C HOLDINGS INC,1.0000000,1.0000000
1080014,45781M101,45781M10,"Innoviva, Inc.",THERAVANCE INC,0.3076923,0.3076923
1548281,G2266G102,G2266G10,Collabrium Japan Acquisition Corp,COLLABRIUM JAPAN ACQUISIT CORP,0.9000000,0.9000000
1464790,05580M108,05580M10,"B. Riley Financial, Inc.",B RILEY FINANCIAL INC,1.0000000,1.0000000
1540729,34552U104,34552U10,Foresight Energy LP,FORESIGHT ENERGY LP,1.0000000,1.0000000
1016838,M81865103,M8186510,RADCOM LTD,RADCOM LTD,1.0000000,1.0000000


In [107]:
to_analyse <- nines_w_stocknames %>% filter(sim_index_max < 0.8) %>% anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))

Joining, by = c("cik", "cusip")


cik,cusip,ncusip,company_name,issuer_name,sim_index_norm,sim_index_max


In [ ]:
to_analyse$cusip_raw <- to_analyse$cusip

In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [109]:
valid8s_above_10 <- cusip_cik %>% group_by(cik, cusip) %>% summarise(freq = n()) %>% ungroup() %>% inner_join(cusip_cik) %>%
    filter(nchar(cusip) == 8 & freq >= 10) %>% collect() %>%
    distinct(cik, cusip, company_name)


valid8s_above_10 <- valid8s_above_10 %>% mutate(ncusip = cusip, cusip6 = substr(cusip, 1, 6))

added_value_8s_above_10 <- valid8s_above_10 %>% anti_join(valid9s_above_10, by = c('cik', 'ncusip'))

added_value_8s_above_10_w_issuers <- added_value_8s_above_10 %>% inner_join(issuers)


Joining, by = c("cik", "cusip")
Joining, by = "cusip6"


In [68]:
dbDisconnect(pg)

[1] TRUE

In [110]:
added_value_8s_above_10_w_issuers %>% head(10)

cik,cusip,company_name,ncusip,cusip6,issuer_check,issuer_name_1,issuer_name_2,issuer_name_3,issuer_adl_1,...,issuer_update_date,cabre_id,cabre_status,lei_cici,legal_entity_name,previous_name,entry_date,cp_institution_type,issuer_name,issuer_adl
750556,69149730,SUNTRUST BANKS INC,69149730,691497,2,OXFORD INDS INC,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,1993-10-31,NA,OXFORD INDS INC,NA
1093367,02932810,AROC INC,02932810,029328,2,AMERICAN RIVS OIL CO,NA,NA,REORGANIZED AS AROC INC,...,NA,NA,NA,NA,NA,NA,1995-12-14,NA,AMERICAN RIVS OIL CO,REORGANIZED AS AROC INC 12/08/1999
1093367,02932810,AMERICAN RIVERS OIL CO /DE/,02932810,029328,2,AMERICAN RIVS OIL CO,NA,NA,REORGANIZED AS AROC INC,...,NA,NA,NA,NA,NA,NA,1995-12-14,NA,AMERICAN RIVS OIL CO,REORGANIZED AS AROC INC 12/08/1999
882291,00808Y09,AETHLON MEDICAL INC,00808Y09,00808Y,0,AETHLON MED INC,NA,NA,NA,...,NA,US000808Y3,A,NA,"Aethlon Medical, Inc.",NA,2000-03-31,NA,AETHLON MED INC,NA
57139,5024709C,LABARGE INC,5024709C,502470,8,LA BARGE INC,NA,NA,NA,...,NA,US05024700,A,NA,LA BARGE INCORPORATED,NA,1993-10-31,NA,LA BARGE INC,NA
1389168,87612G10,"Targa Resources, Inc.",87612G10,87612G,2,TARGA RES CORP,NA,NA,NA,...,NA,US087612G2,A,NA,Targa Resources Corp.,NA,2010-11-09,NA,TARGA RES CORP,NA
1113227,90337P10,CAPSULE COMMUNICATIONS INC DE,90337P10,90337P,0,US WATS INC,NA,NA,REORGANIZED AS CAPSULE,...,NA,NA,NA,NA,NA,NA,1993-10-31,NA,US WATS INC,REORGANIZED AS CAPSULE COMMUNICATIONS INC 04/27/2000
720851,64107410,NESTOR INC,64107410,641074,0,NESTOR INC,NA,NA,NA,...,NA,US06410742,A,NA,"Nestor, Inc.",NA,1993-10-31,NA,NESTOR INC,NA
937252,14166N20,CAREADVANTAGE INC,14166N20,14166N,1,CAREADVANTAGE INC,NA,NA,NA,...,NA,US014166N3,A,NA,"Careadvantage, Inc.",NA,1995-03-30,NA,CAREADVANTAGE INC,NA
811641,45252306,IMMUCELL CORP /DE/,45252306,452523,4,IMMERSION HUMAN INTERFACE CORP,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,1999-12-29,NA,IMMERSION HUMAN INTERFACE CORP,NA


In [173]:
to_analyse <- added_value_8s_above_10_w_issuers %>% anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

cik,cusip,company_name,ncusip,cusip6,issuer_check,issuer_name_1,issuer_name_2,issuer_name_3,issuer_adl_1,...,issuer_update_date,cabre_id,cabre_status,lei_cici,legal_entity_name,previous_name,entry_date,cp_institution_type,issuer_name,issuer_adl


In [ ]:
to_analyse$cusip_raw <- to_analyse$cusip

In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [84]:
added_value_8s_above_10_no_issuer <- added_value_8s_above_10 %>%
                                anti_join(added_value_8s_above_10_w_issuers %>% distinct(cik, cusip))



added_value_8s_above_10_stocknames_only <- added_value_8s_above_10_no_issuer %>%
                                    inner_join(stocknames %>% mutate(cusip6 = substr(ncusip, 1, 6)), by = 'cusip6') %>%
                                    select(cik, cusip.x, company_name, comnam, ncusip.y) %>% 
                                    rename(cusip = cusip.x, ncusip = ncusip.y)



Joining, by = c("cik", "cusip")


In [87]:
added_value_8s_above_10_stocknames_only %>% select(cik, cusip, company_name, comnam, ncusip) %>% arrange(cik, cusip)

cik,cusip,company_name,comnam,ncusip
874516,M2051410,BVR TECHNOLOGIES LTD,B V R TECHNOLOGIES LTD,M2051410
874516,M2051410,BVR TECHNOLOGIES LTD,B V R TECHNOLOGIES LTD,M2051410
928340,G1981E10,CONSOLIDATED WATER CO LTD,CAYMAN WATER CO LTD,G1981E10
928340,G1981E10,CAYMAN WATER CO LTD,CAYMAN WATER CO LTD,G1981E10
1078276,G6116R10,MIH LTD,M I H LTD TORTOLA,G6116R10
1478121,71707104,Pfenex Inc.,PFENEX INC,71707110


In [174]:
to_analyse <- added_value_8s_above_10_stocknames_only %>% anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,comnam,ncusip
1478121,71707104,Pfenex Inc.,PFENEX INC,71707110


In [ ]:
to_analyse$cusip_raw <- to_analyse$cusip

In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [176]:
valid8s_above_10_padded <- valid8s_above_10 %>% rename(cusip_raw = cusip) %>%
    mutate(cusip = paste0('0', cusip_raw)) %>% mutate(ncusip = substr(cusip, 1, 8)) %>%
    mutate(cusip6 = substr(cusip, 1, 6))

added_value_8s_above_10_padded <- valid8s_above_10_padded %>% anti_join(valid9s_above_10, by = c('cik', 'cusip')) %>%
                                    anti_join(valid8s_above_10, by = c('cik', 'ncusip'))


added_value_8s_above_10_padded_w_issuers <- added_value_8s_above_10_padded %>% inner_join(issuers)


Joining, by = "cusip6"


In [177]:
added_value_8s_above_10_padded_w_issuers %>% select(cik, cusip, company_name, issuer_name, issuer_adl, cusip_raw) %>% head(10)

cik,cusip,company_name,issuer_name,issuer_adl,cusip_raw
856716,095825910,WESTERN GAS RESOURCES INC,BLUE RIDGE BANKSHARES INC VA,NA,95825910
1017137,09255Q101,VIALINK CO,BLACKROCK DEFINED OPPORTUNITY CR TR,NA,9255Q101
1002658,023437N10,DALEEN TECHNOLOGIES INC,AMEE INC,NAME CHANGED TO AQUAPLAN INC 12/15/1998 SEE 03839D,23437N10
866873,024522P10,DEL MONTE FOODS CO,AMERICAN BEACON ADVISORS COLLECTIVE INVT TR,NA,24522P10
1324105,08883T200,"Bidz.com, Inc.",BIDZ COM INC,NA,8883T200
921500,020934110,CONSOLIDATED GRAPHICS INC /TX/,ALPHA GROUP INDS INC,NA,20934110
1113481,058468810,MEDICINES CO/ MA,BALI EXPL LTD,NA,58468810
313143,040502410,HAEMONETICS CORP,ARIZONA COMM BK TUCSON,NA,40502410
805037,059523C10,MID ATLANTIC MEDICAL SERVICES INC,BANC AMER FDG 2007-5 TR,NA,59523C10
30547,026609310,DUPLEX PRODUCTS INC,AMERICAN HOME PRODS CORP,NAME CHANGED TO WYETH 03/11/2002 SEE 983024,26609310


In [178]:
to_analyse <- added_value_8s_above_10_padded_w_issuers %>% 
                select(cik, cusip, company_name, issuer_name, issuer_adl, cusip_raw) %>%
                    anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,issuer_adl,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [117]:
added_value_8s_above_10_padded_no_issuers <- added_value_8s_above_10_padded %>%
                                            anti_join(added_value_8s_above_10_padded_w_issuers %>% distinct(cik, cusip))


added_value_8s_above_10_padded_stocknames <- added_value_8s_above_10_padded_no_issuers %>%
                                                inner_join(stocknames, by = 'ncusip')

Joining, by = c("cik", "cusip")


In [118]:
added_value_8s_above_10_padded_stocknames %>% head(10)

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

cik,cusip_unpadded,company_name,ncusip,cusip6,cusip.x,permno,permco,namedt,nameenddt,...,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum


In [180]:
to_analyse <- added_value_8s_above_10_padded_stocknames %>% anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = "cik"
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

cik,cusip_unpadded,company_name,ncusip,cusip6,cusip.x,permno,permco,namedt,nameenddt,...,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [121]:
valid7s_above_10 <- cusip_cik %>% group_by(cik, cusip) %>% summarise(freq = n()) %>% ungroup() %>%
    inner_join(cusip_cik) %>% filter(nchar(cusip) == 7 & freq >= 10) %>% collect() %>%
    distinct(cik, cusip, company_name) %>% rename(cusip_raw = cusip) %>% mutate(cusip = substr(cusip_raw, 1, 6))


valid7s_above_10 <- valid7s_above_10 %>% mutate(cusip6 = substr(cusip,  1, 6))

added_value_7s_above_10 <- valid7s_above_10 %>% anti_join(valid9s_above_10, by = c('cik', 'cusip6')) %>%
                     anti_join(valid8s_above_10, by = c('cik', 'cusip6')) %>%
                     anti_join(valid8s_above_10_padded, by = c('cik', 'cusip6'))


added_value_7s_above_10_w_issuers <- added_value_7s_above_10 %>% inner_join(issuers)


Joining, by = c("cik", "cusip")
Joining, by = "cusip6"


In [181]:
added_value_7s_above_10_w_issuers %>% select(cik, cusip, company_name, issuer_name, issuer_adl, cusip_raw) %>% head(10)

cik,cusip,company_name,issuer_name,issuer_adl,cusip_raw
74702,683836,OPTICAL RADIATION CORP,OPTICAL RADIATION CORP,NA,6838361
863445,066521,BANNER AEROSPACE INC,BANKWEST CORP,NA,0665210
896717,594949,MICROCAP FUND INC,MICROCAP FD INC,NA,5949490
1070304,666114,ORIX CORP,NORTHFIELD BANCORP INC,ACQUIRED BY PATAPSCO BANCORP INC 11/13/2000,6661144
34408,307001,FAMILY DOLLAR STORES INC,FAMILY ED CO INC,NA,3070017
34408,307001,FAMILY DOLLAR STORES INC,FAMILY ED CO INC,NA,3070019
1115837,607409,MOBILE TELESYSTEMS OJSC,MOBILE TELESYSTEMS OJSC,NA,6074091
1115837,607409,MOBILE TELESYSTEMS PJSC,MOBILE TELESYSTEMS OJSC,NA,6074091
809023,012905,BRITISH AIRWAYS PLC,ALBERTA WASTE RES & ENERGY CORP,NA,0129057
876573,71361E,PERCON INC,PERCON ACQUISITION INC,NA,71361E1


In [182]:
to_analyse <- added_value_7s_above_10_w_issuers %>% select(cik, cusip, company_name, issuer_name, issuer_adl, cusip_raw) %>%
                    anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,issuer_adl,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [185]:
added_value_7s_above_10_no_issuers <- added_value_7s_above_10 %>%
                    anti_join(added_value_7s_above_10_w_issuers, by = c('cik', 'cusip'))

added_value_7s_above_10_stocknames <- added_value_7s_above_10_no_issuers %>%
          inner_join(stocknames %>% mutate(cusip6 = substr(ncusip, 1, 6)), by = 'cusip6') %>% 
            rename(cusip = cusip.x, issuer_name = comnam)

In [186]:
added_value_7s_above_10_stocknames

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

cik,cusip_raw,company_name,cusip,cusip6,permno,permco,namedt,nameenddt,cusip.y,...,ticker,issuer_name,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum


In [187]:
to_analyse <- added_value_7s_above_10_stocknames %>% anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

cik,cusip_raw,company_name,cusip,cusip6,permno,permco,namedt,nameenddt,cusip.y,...,ticker,issuer_name,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [129]:
valid7s_above_10 %>% head(5)

cik,cusip_raw,company_name,cusip,cusip6
876573,71361E1,PERCON INC,71361E,71361E
800458,7601742,RENTRAK CORP,760174,760174
723125,5951124,MICRON TECHNOLOGY INC,595112,595112
852952,9196303,WESTPOINT STEVENS INC,919630,919630
1053010,2144177,ROYAL OLYMPIC CRUISE LINES INC,214417,214417


In [136]:
valid7s_above_10_1pad <- valid7s_above_10 %>% 
    mutate(cusip = paste0('0', cusip_raw)) %>% mutate(cusip6 = substr(cusip,  1, 6), ncusip = cusip)

added_value_7s_above_10_1pad <- valid7s_above_10_1pad %>% anti_join(valid9s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10_padded, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10, by = c('cik', 'cusip6'))


added_value_7s_above_10_1pad_w_issuers <- added_value_7s_above_10_1pad %>% inner_join(issuers)


Joining, by = "cusip6"


In [188]:
added_value_7s_above_10_1pad_w_issuers %>% select(cik, cusip, company_name, issuer_name, issuer_adl, cusip_raw) %>% head(10)

cik,cusip,company_name,issuer_name,issuer_adl,cusip_raw
1116581,05982200,DIALOG SEMICONDUCTOR PLC,BANDAI LTD,NA,5982200
809023,00129057,BRITISH AIRWAYS PLC,AHI HEALTHCARE SYS INC,NA,0129057
932290,08860271,THRUSTMASTER INC,BFP RECEIVABLES CORP,NA,8860271
932290,08860271,CENTERSPAN COMMUNICATIONS CORP,BFP RECEIVABLES CORP,NA,8860271
875159,02283401,XL CAPITAL LTD,AMALGATED TECHNOLOGIES PLC,NA,2283401
34408,03070017,FAMILY DOLLAR STORES INC,AMERIFIRST BANK FED SVGS BK MIAMI FLA,FORMERLY AMERIFIRST FED SVGS & LN ASSN MIAMI FLA 03/01/1988,3070017
34408,03070019,FAMILY DOLLAR STORES INC,AMERIFIRST BANK FED SVGS BK MIAMI FLA,FORMERLY AMERIFIRST FED SVGS & LN ASSN MIAMI FLA 03/01/1988,3070019
723125,05951124,MICRON TECHNOLOGY INC,BANC AMER COML MTG TR 2007-2,NA,5951124
74702,06838361,OPTICAL RADIATION CORP,BARRE TR CO VT,MERGED INTO MERCHANTS NATL BK BURLINGTON VT 07/1970,6838361
924613,05902941,NOKIA CORP,BALTIMORE BANCORP,NA,5902941


In [138]:
cusip_cik_exceptions %>% filter(cik == 1116581)

cik,cusip,company_name,issuer_name,issuer_adl,company_name_norm,issuer_name_norm,sim_index_norm,sim_index_max,valid_match,...,better_cusip_issuer_name,better_cusip_issuer_adl,sim_ix_better_cusip,better_cusip6,better_cusip6_issuer_name,better_cusip6_issuer_adl,better_cusip8,better_cusip8_comnam,other_reason,cusip_raw
1116581,05982200,DIALOG SEMICONDUCTOR PLC,BANDAI LTD,NA,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,5982200


In [193]:
to_analyse <- added_value_7s_above_10_1pad_w_issuers %>%
            select(cik, cusip, company_name, issuer_name, issuer_adl, cusip_raw) %>%
            anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,issuer_adl,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [140]:
added_value_7s_above_10_1pad_no_issuers <- added_value_7s_above_10_1pad %>%
                                anti_join(added_value_7s_above_10_1pad_w_issuers, by = c('cik', 'cusip'))


added_value_7s_above_10_1pad_stocknames <- added_value_7s_above_10_1pad_no_issuers %>%
                                inner_join(stocknames %>% mutate(cusip6 = substr(ncusip, 1, 6)), by = 'cusip6')

In [196]:
added_value_7s_above_10_1pad_stocknames

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

cik,cusip_raw,company_name,cusip.x,cusip6,ncusip.x,permno,permco,namedt,nameenddt,...,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum


In [198]:
to_analyse <- added_value_7s_above_10_1pad_stocknames %>% rename(issuer_name = comnam, cusip = cusip.x) %>%
            select(cik, cusip, company_name, issuer_name, cusip_raw) %>%
            anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [146]:
valid7s_above_10_2pad <- valid7s_above_10 %>%  mutate(cusip = paste0('00', cusip_raw)) %>%
    mutate(cusip6 = substr(cusip,  1, 6), ncusip = substr(cusip, 1, 8))

added_value_7s_above_10_2pad <- valid7s_above_10_2pad %>% anti_join(valid9s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10_padded, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10_1pad, by = c('cik', 'cusip6'))


added_value_7s_above_10_2pad_w_issuers <- added_value_7s_above_10_2pad %>% inner_join(issuers)


Joining, by = "cusip6"


In [145]:
cusip_cik_exceptions %>% filter(cik == 1094005)

cik,cusip,company_name,issuer_name,issuer_adl,company_name_norm,issuer_name_norm,sim_index_norm,sim_index_max,valid_match,...,better_cusip_issuer_name,better_cusip_issuer_adl,sim_ix_better_cusip,better_cusip6,better_cusip6_issuer_name,better_cusip6_issuer_adl,better_cusip8,better_cusip8_comnam,other_reason,cusip_raw
1094005,002579230,SINA CORP,ABACUS 2006-17 LTD,NA,NA,NA,NA,NA,FALSE,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,2579230


In [147]:
added_value_7s_above_10_2pad_w_issuers %>% select(cik, cusip, company_name, issuer_name, issuer_adl) %>% head(10)

cik,cusip,company_name,issuer_name,issuer_adl
1094005,002579230,SINA CORP,ABACUS 2006-17 LTD,NA
876573,0071361E1,PERCON INC,ADMIRAL CBO CAYMAN LTD / ADMIRAL CBO DEL INC,NA
353278,007077524,NOVO NORDISK A S,ADLER ELECTRONICS INC,NA
1022259,0074157E1,PRIME SERVICE INC,ADVANCE COSMETIC & ANTI-AGING CTR INC,NA
1263043,006397502,SHINHAN FINANCIAL GROUP CO LTD,ADAMS STR CBO LTD,NA
844843,003736541,GEOTEK COMMUNICATIONS INC,ABOUT COM INC,NA
823546,00088861L,B4MC GOLD MINES INC,ADC TELECOMMUNICATIONS LONG TERM EQUITY OPTIONS-YTL-KTL,NA
1207179,002367963,GOLAR LNG LTD,AVI HLDGS INC,NA
800458,007601742,RENTRAK CORP,AEGEAN MARINE CORP,NA
77597,008472201,SPARTECH CORP,AGILE FDS INC,NA


In [199]:
to_analyse <- added_value_7s_above_10_2pad_w_issuers %>% 
                select(cik, cusip, company_name, issuer_name, issuer_adl, cusip_raw) %>%
                    anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,issuer_adl,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [149]:
added_value_7s_above_10_2pad_no_issuers <- added_value_7s_above_10_2pad %>%
                                    anti_join(added_value_7s_above_10_2pad_w_issuers)

added_value_7s_above_10_2pad_stocknames <- added_value_7s_above_10_2pad_no_issuers %>%
                                        inner_join(stocknames %>% mutate(cusip6 = substr(ncusip, 1, 6)), by = 'cusip6')

Joining, by = c("cik", "cusip_raw", "company_name", "cusip", "cusip6", "ncusip")


In [150]:
added_value_7s_above_10_2pad_stocknames

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

cik,cusip_raw,company_name,cusip.x,cusip6,ncusip.x,permno,permco,namedt,nameenddt,...,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum


In [200]:
to_analyse <- added_value_7s_above_10_2pad_stocknames %>% rename(issuer_name = comnam, cusip = cusip.x) %>%
            select(cik, cusip, company_name, issuer_name, cusip_raw) %>%
            anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [155]:
valid6s_above_10 <- cusip_cik %>% group_by(cik, cusip) %>% summarise(freq = n()) %>% ungroup() %>%
    inner_join(cusip_cik) %>% filter(nchar(cusip) == 6 & freq >= 10) %>% collect() %>%
    distinct(cik, cusip, company_name) 

valid6s_above_10 <- valid6s_above_10 %>% mutate(cusip6 = cusip)


added_value_6s_above_10 <- valid6s_above_10 %>% anti_join(valid9s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10_padded, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10_1pad, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10_2pad, by = c('cik', 'cusip6')) %>% mutate(cusip_raw = cusip)


added_value_6s_above_10_w_issuers <- added_value_6s_above_10 %>% inner_join(issuers) 


Joining, by = c("cik", "cusip")
Joining, by = "cusip6"


In [202]:
added_value_6s_above_10_w_issuers %>% select(cik, cusip, company_name, issuer_name, issuer_adl, cusip_raw) %>% head(10)

cik,cusip,company_name,issuer_name,issuer_adl,cusip_raw
720875,268157,DYNATRONICS CORP,DYNATRONICS CORP,NA,268157
1445625,65338J,NEXAIRA WIRELESS INC.,NEXAIRA WIRELESS INC,NA,65338J
811808,86606G,SUMMIT FINANCIAL GROUP INC,SUMMIT FINL GROUP INC,NA,86606G
1024048,436938,HOMELIFE INC,HOMELIFE INC,NAME CHANGED TO MONEYLOGIX GROUP INC 01/29/2008 SEE 60935X,436938
1355242,050751,"Audio Stocks, Inc.",AUDIOSTOCKS INC,NAME CHANGED TO SHRINK NANOTECHNOLOGIES INC 05/14/2009 SEE 82554U,050751
101265,901637,UNITED ILLUMINATING CO,TWIN GOLD MINES LTD,NA,901637
914791,025164,LUCOR INC /FL/,AMERICAN COLD LITE TECHNOLOGIES INC,NA,025164
1109141,687082,OROMIN EXPLORATIONS LTD,OROMIN EXPLORATIONS LTD NEW,NA,687082
736952,144105,TUXIS CORP,CAROLINA PAC PLYWOOD INC,ACQUIRED BY SOUTHWEST FOREST INDS INC 10/06/1972,144105
1099066,046352,ASTRALIS LTD,ASTRALIS LTD,FORMERLY ASTRALIS PHARMACEUTICALS LTD 12/10/2001,046352


In [201]:
to_analyse <- added_value_6s_above_10_w_issuers %>% select(cik, cusip, company_name, issuer_name, issuer_adl, cusip_raw) %>%
                    anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,issuer_adl,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [158]:
added_value_6s_above_10_no_issuers <- added_value_6s_above_10 %>% anti_join(added_value_6s_above_10_w_issuers)


added_value_6s_above_10_stocknames <- added_value_6s_above_10_no_issuers %>%
                                    inner_join(stocknames %>% mutate(cusip6 = substr(ncusip, 1, 6)), by = 'cusip6')

Joining, by = c("cik", "cusip", "company_name", "cusip6", "cusip_raw")


In [160]:
added_value_6s_above_10_stocknames 

cik,cusip.x,company_name,cusip6,cusip_raw,permno,permco,namedt,nameenddt,cusip.y,...,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum
1396016,G32335,Exceed Co Ltd.,G32335,G32335,92507,52850,2009-10-21,2015-04-29,G3233510,...,EDS,EXCEED COMPANY LTD,3,3,6799,12,NA,2007-12-31,2015-04-30,2


In [161]:
added_value_6s_above_10_stocknames %>% select(cik, cusip.x, company_name, comnam, cusip_raw) %>% 
                                        rename(cusip = cusip.x, issuer_name = comnam)

cik,cusip,company_name,issuer_name,cusip_raw
1396016,G32335,Exceed Co Ltd.,EXCEED COMPANY LTD,G32335


In [204]:
to_analyse <- added_value_6s_above_10_stocknames %>% select(cik, cusip.x, company_name, comnam, cusip_raw) %>% 
                rename(cusip = cusip.x, issuer_name = comnam) %>% anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [209]:
valid6s_above_10_1pad <- valid6s_above_10 %>% rename(cusip_raw = cusip) %>%
    mutate(cusip = substr(paste0('0', cusip_raw), 1, 6)) %>% mutate(cusip6 = substr(cusip,  1, 6))

added_value_6s_above_10_1pad <- valid6s_above_10_1pad %>% anti_join(valid9s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10_padded, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10_1pad, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10_2pad, by = c('cik', 'cusip6')) %>%
    anti_join(added_value_6s_above_10, by = c('cik', 'cusip6'))


added_value_6s_above_10_1pad_w_issuers <- added_value_6s_above_10_1pad %>% inner_join(issuers)


Joining, by = "cusip6"


In [212]:
added_value_6s_above_10_1pad_w_issuers %>% select(cik, cusip, company_name, issuer_name, cusip_raw)

cik,cusip,company_name,issuer_name,cusip_raw
792977,000753,ADVANCED MAGNETICS INC,ABD AMERN CAP MKTS FDS INC,00753P
852570,001787,GLOBAL AXCESS CORP,AMS-3 2003 LP,017874
806514,001640,ADVANCED MATERIALS GROUP INC,AMCA INTL LTD,016401
806514,001640,"Advanced Materials Group, Inc.",AMCA INTL LTD,016401
1113423,000797,Aeterna Zentaris Inc.,ACE SECS CORP HOME EQUITY LN TR SER 2007- HE5,007975
1113423,000797,AETERNA LABORATORIES INC,ACE SECS CORP HOME EQUITY LN TR SER 2007- HE5,007975
1036848,000773,AEROCENTURY CORP,ABKA INC,007737
726958,014752,CASEYS GENERAL STORES INC,ALEXANDERS INC,147528
848865,001958,ALLIED WASTE INDUSTRIES INC,AT/COMM INC,019589
36840,033614,FIRST REAL ESTATE INVESTMENT TRUST OF NEW JERSEY,ANDERSON COMPUTERS / TIDALWAVE CORP,336142


In [211]:
to_analyse <- added_value_6s_above_10_1pad_w_issuers %>% select(cik, cusip, company_name, issuer_name, cusip_raw) %>% 
                anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [214]:
added_value_6s_above_10_1pad_no_issuers <- added_value_6s_above_10_1pad %>%
                                            anti_join(added_value_6s_above_10_1pad_w_issuers)


added_value_6s_above_10_1pad_stocknames <- added_value_6s_above_10_1pad_no_issuers %>%
    inner_join(stocknames %>% mutate(cusip6 = substr(ncusip, 1, 6)), by = 'cusip6')

Joining, by = c("cik", "cusip_raw", "company_name", "cusip6", "cusip")


In [216]:
to_analyse <- added_value_6s_above_10_1pad_stocknames %>% select(cik, cusip.x, company_name, comnam, cusip_raw) %>% 
                rename(cusip = cusip.x, issuer_name = comnam) %>% anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [220]:
valid6s_above_10_2pad <- valid6s_above_10 %>% rename(cusip_raw = cusip) %>%
    mutate(cusip = paste0('00', cusip_raw)) %>% mutate(cusip6 = substr(cusip,  1, 6), ncusip = cusip)


added_value_6s_above_10_2pad <- valid6s_above_10_2pad %>% anti_join(valid9s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10_padded, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10_1pad, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10_2pad, by = c('cik', 'cusip6')) %>%
    anti_join(added_value_6s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(added_value_6s_above_10_1pad, by = c('cik', 'cusip6'))




added_value_6s_above_10_2pad_w_issuers <- added_value_6s_above_10_2pad %>% inner_join(issuers)


Joining, by = "cusip6"


In [221]:
to_analyse <- added_value_6s_above_10_2pad_w_issuers %>% select(cik, cusip, company_name, issuer_name, cusip_raw) %>% 
                anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [223]:
added_value_6s_above_10_2pad_no_issuers <- added_value_6s_above_10_2pad %>%
    anti_join(added_value_6s_above_10_2pad_w_issuers)


added_value_6s_above_10_2pad_stocknames <- added_value_6s_above_10_2pad_no_issuers %>%
    inner_join(stocknames %>% mutate(cusip6 = substr(ncusip, 1, 6)), by = 'cusip6')

Joining, by = c("cik", "cusip_raw", "company_name", "cusip6", "cusip", "ncusip")


In [224]:
added_value_6s_above_10_2pad_stocknames %>% head(5)

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

cik,cusip_raw,company_name,cusip6,cusip.x,ncusip.x,permno,permco,namedt,nameenddt,...,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum


In [225]:
to_analyse <- added_value_6s_above_10_2pad_stocknames %>% select(cik, cusip.x, company_name, comnam, cusip_raw) %>% 
                rename(cusip = cusip.x, issuer_name = comnam) %>% anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [226]:
valid6s_above_10_3pad <- valid6s_above_10 %>% rename(cusip_raw = cusip) %>%
    mutate(cusip = paste0('000', cusip_raw)) %>% mutate(cusip6 = substr(cusip,  1, 6), ncusip = substr(cusip, 1, 8))


added_value_6s_above_10_3pad <- valid6s_above_10_3pad %>% anti_join(valid9s_above_10, by = c('cik', 'cusip')) %>%
    anti_join(valid8s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid8s_above_10_padded, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10_1pad, by = c('cik', 'cusip6')) %>%
    anti_join(valid7s_above_10_2pad, by = c('cik', 'cusip6')) %>%
    anti_join(added_value_6s_above_10, by = c('cik', 'cusip6')) %>%
    anti_join(added_value_6s_above_10_1pad, by = c('cik', 'cusip6')) %>%
    anti_join(added_value_6s_above_10_2pad, by = c('cik', 'cusip6'))


added_value_6s_above_10_3pad_w_issuers <- added_value_6s_above_10_3pad %>% inner_join(issuers)


Joining, by = "cusip6"


In [227]:
to_analyse <- added_value_6s_above_10_3pad_w_issuers %>% select(cik, cusip, company_name, issuer_name, cusip_raw) %>% 
                anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")

In [228]:
added_value_6s_above_10_3pad_no_issuers <- added_value_6s_above_10_3pad %>%
    anti_join(added_value_6s_above_10_3pad_w_issuers)


added_value_6s_above_10_3pad_stocknames <- added_value_6s_above_10_3pad_no_issuers %>%
    inner_join(stocknames %>% mutate(cusip6 = substr(ncusip, 1, 6)), by = 'cusip6')

Joining, by = c("cik", "cusip_raw", "company_name", "cusip6", "cusip", "ncusip")


In [230]:
to_analyse <- added_value_6s_above_10_pad_stocknames %>% select(cik, cusip.x, company_name, comnam, cusip_raw) %>% 
                rename(cusip = cusip.x, issuer_name = comnam) %>% anti_join(cusip_cik_exceptions %>% distinct(cik, cusip))
to_analyse

Joining, by = c("cik", "cusip")


cik,cusip,company_name,issuer_name,cusip_raw


In [ ]:
cols <- colnames(cusip_cik_exceptions)[colnames(cusip_cik_exceptions) %in% colnames(to_analyse)]
cols

In [ ]:
row_cik <- 
row_cusip <- 

df <- to_analyse[to_analyse$cik == row_cik & to_analyse$cusip == row_cusip, cols]

In [ ]:
# See which other cusips have matched row_cik and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cik == row_cik)

In [ ]:
# See which other ciks have matched row_cusip and which ones are better matches
valid9s_above_10_w_issuers %>% filter(cusip == row_cusip)

In [ ]:
# Search for names in edgar.filings by cik
name_search_sql <- paste0('SELECT DISTINCT cik, company_name FROM edgar.filings WHERE cik = ', row_cik)
tbl(pg, sql(name_search_sql))

In [ ]:
# Search for ciks in edgar.filings by using first few words of a company name
company_name_str <- ""
cik_search_sql <- paste0("SELECT DISTINCT cik, company_name FROM edgar.filings WHERE UPPER(company_name) ~ '^", 
                         company_name_str, "'")
tbl(pg, sql(cik_search_sql))

In [ ]:
# Try looking for cusip6's in issuers by searching by first few words of company_name
company_name_str <- ''
issuers %>% filter(grepl(paste0('^', company_name_str), issuer_name))

In [ ]:
# Try looking for cusip8's in stocknames by searching by first few words of company_name
company_name_str <- ''
stocknames %>% filter(grepl(paste0('^', company_name_str), comnam))

In [ ]:
df$valid_match <- 
df$better_cik <- 
df$better_cik_company_name <- 
df$better_cusip <- 
df$better_cusip_issuer_name <-
df$better_cusip_issuer_adl <- 
df$better_cusip6 <- 
df$better_cusip6_issuer_name <-
df$better_cusip6_issuer_adl <-
df$better_cusip8 <- 
df$better_cusip8_comnam <- 
df$other_reason <- 

In [ ]:
rs <- dbWriteTable(pg, c('edgar', 'cusip_cik_exceptions'), df, row.names = FALSE, append = TRUE,
                   encoding = "latin1")